# Electrochemically Active Surface Area (ECSA) Calculation
Linn Kelley<br>
08/13/2022<br>

This script calculates ECSA for a catalyst using the hydrogen underpotential deposition (H<sub>upd</sub>) method.

Prior to onset of HER (at potentials positive of the equilibrium voltage), protons are adsorbed onto Pt catalyst surface and reduced to form a metal-hydrogen bond:
<center>
    $H^{+}+M+e^{-}\rightleftharpoons MH_{ads}$
</center>
To find the electrochemically active surface area of a Pt catalyst, a CV from a potential just positive of the standard reduction potential of H<sup>+</sup> (0 V vs RHE) to 1 V will show peaks corresponding to the adsorption and desorption of H<sup>+</sup> on Pt. The adsorption peak corresponds to a negative reduction current and the desorption peak corresponds to a positive oxidation current. The total charge of adsorbed ions can be calculated by integrating the adsorption peak current with respect to time:
<center>
$Q=\int Idt$
</center>
The literature value of surface charge density for H<sup>+</sup> on Pt is 210 µC/cm<sup>2</sup> which is used to convert the integrated charge to ECSA. The CV is collected with H<sub>2</sub> flowed to the anode and inert N<sub>2</sub> flowed to the cathode so the ORR will not interfere with the adsorption and desorption peaks. All CVs have a capacitive baseline current which is current due to the non-Faradaic process of double layer charging. This current should not be included while integrating. The adsorption peak is often larger than the desorption peak due to a small amount of HER onset (reduction of H<sup>+</sup> to form H<sub>2</sub>) and is the more accurate choice for calculating ECSA. It is important to bound the adsorption peak integration prior to the onset of HER. 
<img src="ECSA integration.png" width="600">
This technique is best used for measuring the surface area of Pt-only catalysts; however, it can still be used for Pt-alloy catalysts. The presence of an alloy in the catalyst will make the characteristic peaks of H<sup>+</sup> adsorption less distinct, and the boundary between H<sup>+</sup> adsorption and HER can be hard to recognize. In this case it is best to set the integration boundary at a certain voltage, usually 80 mV. 


### Import libraries

In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import simpledialog
from scipy.signal import argrelextrema
%matplotlib qt

### Import file

In [16]:
ecsa = pd.read_csv('Sample ECSA file.txt',delimiter='\t',header=14)

### Add a time column to the dataframe 
If your file does not have a time column, use the addtime function. 
addtime has two inputs, a dataframe (df) and the interval at which data points were collected (nu). The data collection rate can be found by opening your text file in Notepad and dividing 1/Waveform Rate (Hz).

In [17]:
def addtime(df,nu):
    time = []
    for i in range(len(df)):
        t=i*nu
        time.append(t)
    df['Time']=time 
        
addtime(ecsa,0.01)

### ecsa_integration inputs
Inputs to ECSA function<br>
df: dataframe with a voltage, current, and time column<br>
rng: indices of the last CV cycle in the format: list(range(start_index, end_index)). Use the plot below to find the indices of the desired cycle<br>
pos_l: text with calculated ECSA will appear on the plot, this is the y-position of the bottom of the text<br>
title: title of the plot and how the image will be saved<br>
cell_area: cm<sup>2</sup><br>
CL: catalyst loading, mg<sub>Pt</sub>/cm<sup>2</sup>

The function requires several user inputs. The first is point mapping in a pop-up window; user will select four points:<br>
1. Left of the desorption peak<br>
2. Right of the desorption peak<br>
3. Right of the adsorption peak<br>
4. Left of the adsorption peak<br>

<img src="ECSA mapping points.png" width="600">

When the mapping image pops up, the magnifying glass can be used to zoom in as needed while selecting points. The home button will bring user back to the original image. Hover the mouse over the desired point and press the space bar to select a point. A red X will appear on the point. To erase the point and try again, press the backspace or delete key. When all four points have been selected, press the enter key to continue. 

The next input allows the user to decide if they would like to bound the integration at a certain voltage or let the function detect the edge of the peak. It is recommended to specify a voltage for alloyed catalysts where the adsorption region does not have a distinct boundary with the HER. If the user selects to specify a voltage, they will then be asked to specify a voltage limit for both the upper and lower integrations. The final input allows the user to decide whether or not to plot mapped points. 

### Plot to find indices

In [18]:
start_index = 13801
end_index = 18400
plt.plot(ecsa['Volts'][start_index:end_index],ecsa['Amps'][start_index:end_index])

### ecsa_integration function

In [19]:
def ecsa_integration(df,rng,pos_l,title,cell_area,CL):
    # CV data is plotted and user specifies integration points.
    x_ecsa = np.array(df['Volts'][rng].tolist()) 
    y_ecsa = np.array(df['Amps'][rng].tolist())*(1000/cell_area) 
    t_ecsa = np.array(df['Time'][rng].tolist())        
    mins = np.array(argrelextrema(y_ecsa, np.less)).T 
    maxs = np.array(argrelextrema(y_ecsa, np.greater)).T 
    plt.plot(x_ecsa,y_ecsa)
    pts = plt.ginput(n=-1,timeout=-1,mouse_add=None,mouse_pop=None,mouse_stop=None)
    
    # The data indicies closest to the selected points are recorded in the variable ind.
    ind = []
    points = list(zip(x_ecsa,y_ecsa))
    def distance(a,b):
        return(sum([(k[0]-k[1])**2 for k in zip(a,b)])**0.5)
    for i,pt in enumerate(pts):
        dists = [distance([pt[0], pt[1]],k) for k in points]
        ind.append(dists.index(min(dists)))
    
    # User specifies integration preference.
    ROOT = tk.Tk()
    ROOT.withdraw()
    A = simpledialog.askstring(title="Test",prompt="Set left integration bound at a specified voltage or edge of first peak? (v/e)")
    if A=='v':
        ROOT = tk.Tk()
        ROOT.withdraw()
        v_up = simpledialog.askfloat(title="Test",prompt="Specify voltage for upper integration (V, do not add units)")
        v_down = simpledialog.askfloat(title="Test",prompt="Specify voltage for lower integration (V, do not add units)")
        up_d = np.where(np.abs(np.subtract(x_ecsa[ind[0]:ind[1]],v_up))==min(np.abs(np.subtract(x_ecsa[ind[0]:ind[1]],v_up))))[0][0]+ind[0]
        d_d = np.where(np.abs(np.subtract(x_ecsa[ind[2]:ind[3]],v_down))==min(np.abs(np.subtract(x_ecsa[ind[2]:ind[3]],v_down))))[0][0]+ind[2]
    if A=='e':
        up_d = int(mins[0])
        d_d = int(maxs[-1])
    
    # Integration regions are defined. 
    up_l = np.where(y_ecsa==min(y_ecsa[ind[0]:ind[1]]))[0][0]
    d_l = np.where(y_ecsa==max(y_ecsa[ind[2]:ind[3]]))[0][0]
    upx = [x_ecsa[up_d],x_ecsa[up_l]] 
    upy = [y_ecsa[up_l],y_ecsa[up_l]] 
    dx = [x_ecsa[d_l],x_ecsa[d_d]] 
    dy = [y_ecsa[d_l],y_ecsa[d_l]]
    x_up = x_ecsa[up_d:up_l+1] 
    y_up = y_ecsa[up_d:up_l+1] 
    t_up = t_ecsa[up_d:up_l+1] 
    x_d = x_ecsa[d_l:d_d+1] 
    y_d = y_ecsa[d_l:d_d+1]
    t_d = t_ecsa[d_l:d_d+1]
    line_up = np.interp(x_up,upx,upy) 
    line_d = np.interp(x_d,dx,dy) 

    # Integration regions are outlined and colored. 
    plt.figure()
    plt.plot(x_ecsa,y_ecsa,'dodgerblue')
    plt.fill_between(x_up,line_up,y_up,color='firebrick')
    plt.fill_between(x_d,line_d,y_d,color='gold')
    plt.plot(x_up,line_up,'k')
    plt.plot(x_d,line_d,'k')
    plt.vlines(upx[0],upy[0],y_ecsa[up_d],'k')
    plt.vlines(dx[1],dy[0],y_ecsa[d_d],'k')
    plt.plot(x_up,y_up,'k')
    plt.plot(x_d,y_d,'k')
    plt.plot(x_ecsa[up_d],y_ecsa[up_d],'x',color='mediumblue')
    plt.plot(x_ecsa[up_l],y_ecsa[up_l],'x',color='mediumblue')
    plt.plot(x_ecsa[d_d],y_ecsa[d_d],'x',color='mediumblue')
    plt.plot(x_ecsa[d_l],y_ecsa[d_l],'x',color='mediumblue')
    plt.xlabel('Potential (V vs SHE)')
    plt.ylabel('Current Density (mA/cm$^2$)')
    plt.title(title)
    
    # User specifies mapped points preference. 
    ROOT = tk.Tk()
    ROOT.withdraw()
    a = simpledialog.askstring(title="Test",prompt="Do you want to plot mapped points? (Yes/No):")
    
    if (a=='Yes') or (a=='yes'):
        plt.plot(x_ecsa[ind[0]],y_ecsa[ind[0]],'*',color='k')
        plt.plot(x_ecsa[ind[1]],y_ecsa[ind[1]],'*',color='k')
        plt.plot(x_ecsa[ind[2]],y_ecsa[ind[2]],'*',color='k')
        plt.plot(x_ecsa[ind[3]],y_ecsa[ind[3]],'*',color='k')
        if A=='v':
            plt.axvline(x=v_up)
            plt.axvline(x=v_down)
    
    # Peaks are integrated and the figure is saved.
    g_pt = CL*cell_area/1000
    a_upper = (np.trapz(y_up,t_up) - np.trapz(line_up,t_up))/(1000*210E-6) # cm^2 Pt/cm^2
    a_lower = np.abs((np.trapz(y_d,t_d) - np.trapz(line_d,t_d))/(1000*210E-6))
    ma_upper = a_upper*cell_area/(10000*g_pt)# m^2 Pt/g Pt
    ma_lower = a_lower*cell_area/(10000*g_pt)
    plt.text(0.2,pos_l,'ECSA upper: ' + str(round(ma_upper,2)) + ' m$^2_{Pt}$/g$_{Pt}$ \nECSA upper: ' + str(round(a_upper,2)) + ' cm$^2_{Pt}$/cm$^2$ \nECSA lower: ' + str(round(ma_lower,2)) + ' m$^2_{Pt}$/g$_{Pt}$ \nECSA lower: ' + str(round(a_lower,2)) + ' cm$^2_{Pt}$/cm$^2$')
    plt.savefig(title+'.svg',format='svg')

### Example use of ecsa_integration function

In [20]:
ecsa_integration(ecsa,list(range(13801,18400)),-28,'ECSA test',50,0.253)

Happy integrating! 